##Tirmzi Analysis
n=1000 m+=1000 nm-=120 istep= 4 min=150 max=700

In [1]:
import sys
sys.path
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import signal

In [2]:
ls

 Volume in drive C is Local Disk
 Volume Serial Number is AA1B-997A

 Directory of C:\Users\emaro\OneDrive - University of Mount Union\XSEDE Summer 2021\xsede-spm\data

08/05/2021  10:33 AM    <DIR>          .
08/05/2021  10:33 AM    <DIR>          ..
08/05/2021  10:33 AM    <DIR>          .ipynb_checkpoints
08/02/2021  02:01 PM    <DIR>          0309-VariedNuni_8_2_2021-0000
08/02/2021  02:15 PM    <DIR>          8_2_2021
08/02/2021  03:39 PM           161,361 8_2_2021_FortranErequals1.ipynb
08/03/2021  01:17 PM           130,779 8_2_2021_PythonErequals1.ipynb
08/02/2021  02:58 PM           130,726 8_2_2021_PythonVariedNuni.ipynb
08/04/2021  02:42 PM           130,779 8_2_2021_PythonVariedNuni-Copy3.ipynb
08/02/2021  02:52 PM    <DIR>          8_2_2021P
08/03/2021  03:01 PM           225,696 8_3_2021_SphereTest_Python.ipynb
08/05/2021  10:30 AM            93,725 852021_FDefaultCapsol.ipynb
08/05/2021  10:32 AM            93,725 852021_FOriginalTirmzi.ipynb
08/05/2021  10:33 AM        

In [3]:
import capsol.newanalyzecapsol as ac

In [4]:
ac.get_gridparameters

<function capsol.newanalyzecapsol.get_gridparameters(f_name)>

In [5]:
import glob

In [6]:
folders = glob.glob("Original Tirmzi F/*/")
folders


['Original Tirmzi F\\0335-RR_originalTirmzi-0002\\',
 'Original Tirmzi F\\0337-RR_originalTirmzi-0001\\',
 'Original Tirmzi F\\0339-RR_originalTirmzi-0003\\',
 'Original Tirmzi F\\0341-RR_originalTirmzi-0000\\']

In [7]:
all_data= dict() 
for folder in folders:
    params = ac.get_gridparameters(folder + 'capsol.in')
    data = ac.np.loadtxt(folder + 'Z-U.dat')
    process_data = ac.process_data(params, data, smoothing=False, std=5*10**-9)
    all_data[folder]= (process_data)
all_params= dict()
for folder in folders:
    params=ac.get_gridparameters(folder + 'capsol.in')
    all_params[folder]= (params)

<ipython-input-7-1898c4883aa2>:4: UserWarning: loadtxt: Empty input file: "Original Tirmzi F\0335-RR_originalTirmzi-0002\Z-U.dat"
  data = ac.np.loadtxt(folder + 'Z-U.dat')


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
all_data

In [ ]:
all_data.keys()

In [ ]:
for key in {key: params for key, params in all_params.items() if params['Thickness_sample'] == .5}:
    data=all_data[key]
    thickness =all_params[key]['Thickness_sample']
    rtip= all_params[key]['Rtip']
    er=all_params[key]['eps_r']
    plt.plot(data['z'], data['c'], label= f'{rtip} nm, {er}, {thickness} nm')
    
    
plt.title('C v. Z for 1nm thick sample')    
plt.ylabel("C(m)")
plt.xlabel("Z(m)")
plt.legend()
plt.savefig("C' v. Z for 1nm thick sample 06-28-2021.png")

cut off last experiment because capacitance was off the scale

In [ ]:
for key in {key: params for key, params in all_params.items() if params['Thickness_sample'] == .5}:
    data=all_data[key]
    thickness=all_params[key]['Thickness_sample']
    rtip= all_params[key]['Rtip']
    er=all_params[key]['eps_r']
    s=slice(4,-3)
    plt.plot(data['z'][s], data['cz'][s], label=f'{rtip} nm, {er}, {thickness} nm' )
    
plt.title('Cz vs. Z for 1.0nm')    
plt.ylabel("Cz")
plt.xlabel("Z(m)")
plt.legend()
plt.savefig("Cz v. Z for varying sample thickness, 06-28-2021.png")

In [ ]:
for key in {key: params for key, params in all_params.items() if params['Thickness_sample'] == .5}:
    data=all_data[key]
    thickness=all_params[key]['Thickness_sample']
    rtip= all_params[key]['Rtip']
    er=all_params[key]['eps_r']
    s=slice(5,-5)
    plt.plot(data['z'][s], data['czz'][s], label=f'{rtip} nm, {er}, {thickness} nm' )
    
plt.title('Czz vs. Z for 1.0nm')    
plt.ylabel("Czz")
plt.xlabel("Z(m)")
plt.legend()
plt.savefig("Czz v. Z for varying sample thickness, 06-28-2021.png")

In [ ]:
params

In [ ]:
for key in {key: params for key, params in all_params.items() if params['Thickness_sample'] == .5}:
    data=all_data[key]
    thickness=all_params[key]['Thickness_sample']
    rtip= all_params[key]['Rtip']
    er=all_params[key]['eps_r']
    s=slice(8,-8)
    plt.plot(data['z'][s], data['alpha'][s], label=f'{rtip} nm, {er}, {thickness} nm' )
    
plt.title('alpha vs. Z for 1.0nm')    
plt.ylabel("$\\alpha$")
plt.xlabel("Z(m)")
plt.legend()
plt.savefig("Alpha v. Z for varying sample thickness, 06-28-2021.png")

In [ ]:
data

In [ ]:
from scipy.optimize import curve_fit


In [ ]:
def Cz_model(z, a, n, b,):
    return(a*z**n + b)

In [ ]:
all_data.keys()

In [ ]:
data= all_data['capsol-calc\\0001-capsol\\']
z= data['z'][1:-1]
cz= data['cz'][1:-1]

In [ ]:
popt, pcov= curve_fit(Cz_model, z, cz, p0=[cz[0]*z[0], -1, 0])
a=popt[0]
n=popt[1]
b=popt[2]
std_devs= np.sqrt(pcov.diagonal())
sigma_a = std_devs[0]
sigma_n = std_devs[1]
model_output= Cz_model(z, a, n, b)
rmse= np.sqrt(np.mean((cz - model_output)**2))


In [ ]:
f"a= {a} ± {sigma_a}"

In [ ]:
f"n= {n}± {sigma_n}"

In [ ]:
model_output

In [ ]:
"Root Mean Square Error"

In [ ]:
rmse/np.mean(-cz)